imports +
console timeline rankings + 
functions  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import json

with open('util.txt', 'r') as file:
    line = file.read().strip()

API_KEY = line.split('=')[1].strip()
BASE_URL = 'https://api.rawg.io/api/games'

console_rank = {
    # Gen 1–3 / Classic consoles
    '2600': 1, 'Int': 1, 'CV': 1, 'FDS': 1, 'NES': 2, 'MS': 2, 'SG-1000': 2, 'GB': 3, 'GG': 3, 'SNES': 3,
    'GEN': 3, 'SCD': 3, 'GBC': 3, 'VB': 3, 'TG16': 3, 'PCE': 3, '3DO': 3, 'Jag': 3, 'CDi': 3, 'CD32': 3,

    # Gen 4–5
    'PS': 4, 'N64': 4, 'Saturn': 4, 'DC': 5, 'PS2': 5, 'GC': 5, 'XB': 5, 'GBA': 5,

    # Gen 6–7
    'DS': 6, 'PSP': 6, 'X360': 6, 'PS3': 6, 'Wii': 6,

    # Gen 8
    'PS4': 7, 'XB1': 7, 'XOne': 7, 'WiiU': 7, '3DS': 7, 'PSV': 7,

    # Gen 9
    'Switch': 8, 'NS': 8, 'PS5': 9, 'XSX': 9, 'XS': 9, 'Series': 9,

    # PC & Services
    'PC': 10, 'Linux': 10, 'OSX': 10, 'Mob': 10, 'iOS': 10, 'Android': 10, 'And': 10, 'WinP': 10, 'PSN': 10, 'XBL': 10,

    # Obscure/Other/Unknown
    'Arc': 0, 'All': 0, 'AJ': 0, 'ApII': 0, 'AST': 0, 'Aco': 0, 'Amig': 0, 'BBCM': 0, 'BRW': 0,
    'C64': 0, 'C128': 0, 'DSi': 0, 'DSiW': 0, 'GIZ': 0, 'iQue': 0, 'Lynx': 0,
    'MSD': 0, 'NG': 0, 'NGage': 0, 'OR': 0, 'PCFX': 0, 'S32X': 0, 'VC': 0, 'WS': 0, 'ZXS': 0, 'FMT': 0
}



Functions

In [63]:
def fill_last_update(row, df):
    if pd.notna(row['last_update']):
        return row['last_update']

    same_title = df[df['title'] == row['title']]
    newer_versions = same_title[same_title['console_rank'] > row['console_rank']]
    if not newer_versions.empty:
        # Use the *earliest* release date among newer versions
        replacement_date = newer_versions['release_date'].min()
        return replacement_date

    return row['last_update'] 


def fill_missing_regionals(row):
    if pd.notna(row['total_sales']) and row['genre'] in genre_props.index:
        for region in regional_cols:
            if pd.isna(row[region]):
                proportion = genre_props.loc[row['genre'], region]
                row[region] = row['total_sales'] * proportion
    return row

read csv files

In [ ]:
data_dictionary_df = pd.read_csv('vg_data_dictionary.csv')
sales_data_df = pd.read_csv('vgchartz-2024.csv')

#print(data_dictionary_df.head())
#print(sales_data_df.head())

drop img column and remove it from the data dictionary

In [ ]:

#print(sales_data_df.columns)
sales_data_df = sales_data_df.drop(columns='img')
data_dictionary_df = data_dictionary_df.drop(index=0)
#print(sales_data_df.columns)
#print(data_dictionary_df.head())

Ensure all last_update values are filled.
rules:
1. If rereleased on a newer console the old console's last_update will become the newer console's release date
2. If there is no newer release and the game still has no last_update value then make the last_update = release_date

In [56]:
sales_data_df['console_rank'] = sales_data_df['console'].map(console_rank)
#ensure datetime objects are usable by making them pandas datetime Objects()
sales_data_df['release_date'] = pd.to_datetime(sales_data_df['release_date'], errors='coerce')
sales_data_df['last_update'] = pd.to_datetime(sales_data_df['last_update'], errors='coerce')

sales_data_df['last_update'] = sales_data_df.apply(
    lambda row: fill_last_update(row, sales_data_df), axis=1
)

sales_data_df['last_update'] = sales_data_df['last_update'].fillna(sales_data_df['release_date'])

##print(sales_data_df.head())

                         title console    genre       publisher  \
0           Grand Theft Auto V     PS3   Action  Rockstar Games   
1           Grand Theft Auto V     PS4   Action  Rockstar Games   
2  Grand Theft Auto: Vice City     PS2   Action  Rockstar Games   
3           Grand Theft Auto V    X360   Action  Rockstar Games   
4    Call of Duty: Black Ops 3     PS4  Shooter      Activision   

        developer  critic_score  total_sales  na_sales  jp_sales  pal_sales  \
0  Rockstar North           9.4        20.32      6.37      0.99       9.85   
1  Rockstar North           9.7        19.39      6.06      0.60       9.71   
2  Rockstar North           9.6        16.15      8.41      0.47       5.49   
3  Rockstar North           NaN        15.86      9.06      0.06       5.33   
4        Treyarch           8.1        15.09      6.18      0.41       6.05   

   other_sales release_date last_update  console_rank  
0         3.12   2013-09-17  2014-11-18           6.0  
1         

Now for titles that are still missing a release date I will be using RAWG API to attempt to fill in the missing values. My API calls are limited so this will have limited effectiveness

In [58]:
missing_titles = sales_data_df[sales_data_df['release_date'].isnull()]['title'].unique()
#still have missing values due to no release date. find unique titles to minimize api calls
#values can be filled in later
title_to_date = {}
for title in missing_titles:
    print(title)
    if title in title_to_date:  # Skip if already fetched (in case of re-run)
        continue

    params = {
        'key': API_KEY,
        'search': title,
        'search_exact': True,
        'page_size': 1
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        results = response.json().get('results', [])
        if results:
            release_date = results[0].get('released')
            title_to_date[title] = release_date
        else:
            title_to_date[title] = None
    else:
        print(f"Error fetching '{title}': {response.status_code}")
        title_to_date[title] = None



sales_data_df['release_date'] = sales_data_df.apply(
    lambda row: title_to_date.get(row['title'], row['release_date']) if pd.isnull(row['release_date']) else row['release_date'],
    axis=1
)

missing_count = sales_data_df['release_date'].isnull().sum()
print(f"Missing release_date values: {missing_count}")##let see if this worked

Gran Turismo Concept 2001 Tokyo
TERA
Dragon Ball Z: Budokai Tenkaichi 2 (JP sales)
Custom Robo
Jonah Lomu Rugby Challenge
Danganronpa: Trigger Happy Havoc
Valkyria Chronicles III: Unrecorded Chronicles
Ghostbusters II
Yu Yu Hakusho: Dark Tournament
Famista 64
WRC: FIA World Rally Championship
Sword of the Samurai
Metal Gear Solid: Peace Walker HD Edition
Nintendo Puzzle Collection
GiFTPiA
Atsumare! Power Pro Kun no DS Koushien
Demon Chaos
Zero: Tsukihami no Kamen
Get Fit with Mel B
Mega Man Battle Network: Operation Shooting Star
Kururin Paradise
Mobile Ops: The One Year War
Chou Soujuu Mecha MG
Tour de France 2011
The History Channel: Great Battles - Medieval
B.L.U.E.: Legend of Water
Check vs. Mate
Half-Minute Hero 2
Sega Rally 2006
Fullmetal Alchemist: Brotherhood
PriPara All Idol Perfect Stage!
Bikkuriman Daijiten
Omega Labyrinth Z
Beyond the Labyrinth
Samurai Spirits: Tenkaichi Kenkakuden
Combat Wings: The Great Battles of WWII
Jet Impulse
PDC World Championship Darts 2008
Ferrari

ReadTimeout: HTTPSConnectionPool(host='api.rawg.io', port=443): Read timed out. (read timeout=None)

check to see how many values have been filled/still missing

In [60]:
missing_count = sales_data_df['release_date'].isnull().sum()
print(f"Missing release_date values: {missing_count}")##let see if this worked
print(sales_data_df.isnull().sum())

Missing release_date values: 7051
title               0
console             0
genre               0
publisher           0
developer          17
critic_score    57338
total_sales     45094
na_sales        51379
jp_sales        57290
pal_sales       51192
other_sales     48888
release_date     7051
last_update      1538
console_rank     1390
dtype: int64


Fill in still missing console rank scores. and fill missing critic scores with -1 to show missing

In [61]:
sales_data_df['console_rank'] = sales_data_df['console_rank'].fillna(0)
sales_data_df['critic_score'] = sales_data_df['critic_score'].fillna(-1)
print(sales_data_df.isnull().sum())

title               0
console             0
genre               0
publisher           0
developer          17
critic_score        0
total_sales     45094
na_sales        51379
jp_sales        57290
pal_sales       51192
other_sales     48888
release_date     7051
last_update      1538
console_rank        0
dtype: int64


fill total_sales if missing but regional values exits. Simply, find the sum

Try to approximate regional sales using average proportion of sales per region based on genre


In [64]:
# Only fill total_sales if it's missing but regional values exist
regional_cols = ['na_sales', 'jp_sales', 'pal_sales', 'other_sales']
sales_data_df['total_sales'] = sales_data_df.apply(
    lambda row: row[regional_cols].sum() if pd.isna(row['total_sales']) and row[regional_cols].notna().any() else row['total_sales'],
    axis=1
)

regional_cols = ['na_sales', 'jp_sales', 'pal_sales', 'other_sales']

# Filter to rows with complete data to calculate reliable proportions
valid_sales_df = sales_data_df.dropna(subset=['total_sales'] + regional_cols)

# Compute average proportion of each region relative to total sales per genre
genre_props = valid_sales_df.groupby('genre')[regional_cols].mean().div(
    valid_sales_df.groupby('genre')['total_sales'].mean(), axis=0
)

sales_data_df = sales_data_df.apply(fill_missing_regionals, axis=1)

print(sales_data_df.isnull().sum())

title               0
console             0
genre               0
publisher           0
developer          17
critic_score        0
total_sales     45094
na_sales        45094
jp_sales        45098
pal_sales       45095
other_sales     45094
release_date     7051
last_update      1538
console_rank        0
dtype: int64


compare missing columns to dataFrame shape

In [65]:
print(sales_data_df.shape)

(64016, 14)


I think 19k in data is a solid start. I am still missing plenty of the data in the dataset, but I think this is a solid amount and can still unveil trends and insights.

In [66]:
clean_df = sales_data_df.dropna()
print(f"Clean dataset shape: {clean_df.shape}")

Clean dataset shape: (18825, 14)


Export dataset to csv file

In [67]:
sales_data_df.to_csv('cleaned_video_game_sales.csv', index=False)